In [5]:
import os
import pandas as pd
import torch
from movieparser.scriptparser import ScriptParser
from movieparser.scriptloader import label2id
from IPython.display import display
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from tqdm.notebook import tqdm
from movieparser.parse_scripts_noindent import parse_lines

In [2]:
parser = ScriptParser(38, len(label2id), False)
parser.load_state_dict(torch.load("results/saved_models/epoch1.pt"))
df = pd.read_csv("results/data.csv", index_col=None)
display(df.head())
parser.to(0);

,movie,line_no,text,label,error
0,44_inch_chest,1,out to wear ... whatever combination - it,D,NONE
1,44_inch_chest,2,works! - You look superb! ... And your,D,NONE
2,44_inch_chest,3,underw ear - immac ulate ! 100 % cot ton!,D,NONE
3,44_inch_chest,4,Dazzlin'!... Not like my pinky grey-y,D,NONE
4,44_inch_chest,5,"things! Nah, you've just got it - good at",D,NONE


In [6]:
files = os.listdir("data/SAIL_coref_screenplays/screenplays")
scripts = []

for file in files:
    script = []
    lines = open(os.path.join("data/SAIL_coref_screenplays/screenplays", file)).readlines()
    
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if len(line) == 0:
            j = i + 1
            while j < len(lines) and len(lines[j].strip()) == 0:
                j += 1
            if j < len(lines):
                script.append("")
                script.append(lines[j].strip())
            i = j + 1
        else:
            script.append(line)
            i += 1

    scripts.append(script)

for file, script in tqdm(zip(files, scripts), total=len(files)):
    movie = file.split("/")[-1].split(".")[0]
    parsed_file = os.path.join("data/SAIL_coref_screenplays/robust_parsed", movie + ".csv")
    # tags = parser.parse(script)
    tags = parse_lines(script)
    df = pd.DataFrame()
    df["text"] = script
    df["tags"] = tags
    df.to_csv(parsed_file, index=False)

  0%|          | 0/8 [00:00<?, ?it/s]

In [7]:
df = pd.read_csv("results/data.csv", index_col=None)
display(df.head())
groups = df.groupby(["movie","error"])
print(f"{groups.ngroups} (movie, error) groups")

,movie,line_no,text,label,error
0,44_inch_chest,1,out to wear ... whatever combination - it,D,Original
1,44_inch_chest,2,works! - You look superb! ... And your,D,Original
2,44_inch_chest,3,underw ear - immac ulate ! 100 % cot ton!,D,Original
3,44_inch_chest,4,Dazzlin'!... Not like my pinky grey-y,D,Original
4,44_inch_chest,5,"things! Nah, you've just got it - good at",D,Original


702 (movie, error) groups


In [8]:
len(df)

224260

In [11]:
df.groupby(["movie", "error"]).agg(len).text.values

array([251, 301, 251, 310, 251, 251, 251, 251, 301, 300, 364, 300, 378,
       300, 300, 300, 300, 364, 251, 297, 251, 312, 251, 251, 251, 251,
       297, 344, 422, 344, 435, 344, 344, 344, 344, 422, 250, 296, 250,
       310, 250, 250, 250, 250, 296, 334, 394, 334, 406, 334, 334, 334,
       334, 394, 251, 306, 251, 312, 251, 251, 251, 251, 306, 276, 339,
       276, 349, 276, 276, 276, 276, 339, 251, 302, 251, 307, 251, 251,
       251, 251, 302, 372, 443, 372, 453, 372, 372, 372, 372, 443, 251,
       298, 251, 310, 251, 251, 251, 251, 298, 338, 410, 338, 416, 338,
       338, 338, 338, 410, 250, 290, 250, 299, 250, 250, 250, 250, 290,
       381, 452, 381, 460, 381, 381, 381, 381, 452, 250, 296, 250, 302,
       250, 250, 250, 250, 296, 394, 482, 394, 492, 394, 394, 394, 394,
       482, 250, 302, 250, 322, 250, 250, 250, 250, 302, 338, 398, 338,
       409, 338, 338, 338, 338, 398, 251, 298, 251, 303, 251, 251, 251,
       251, 298, 367, 434, 367, 444, 367, 367, 367, 367, 434, 25

In [14]:
df["error"].unique()

array(['Original', 'Original+NameContainsSceneKeyword',
       'Original+NameContainsTransitionKeyword',
       'Original+SceneHeaderKeywordRemoved',
       'Original+SceneHeaderLowercased',
       'Original+CharacterNameLowercased', 'Original+Watermark',
       'Original+AsterisksNumbers', 'Original+DialogueExpressions',
       'NoEmptyLines', 'NoEmptyLines+NameContainsSceneKeyword',
       'NoEmptyLines+NameContainsTransitionKeyword',
       'NoEmptyLines+SceneHeaderKeywordRemoved',
       'NoEmptyLines+SceneHeaderLowercased',
       'NoEmptyLines+CharacterNameLowercased', 'NoEmptyLines+Watermark',
       'NoEmptyLines+AsterisksNumbers',
       'NoEmptyLines+DialogueExpressions'], dtype=object)

In [18]:
for text in df["text"].fillna("").unique():
    if not isinstance(text, str):
        print(text)

In [22]:
feats_df = pd.read_csv("results/feats.csv", index_col=None).fillna("")

In [23]:
(feats_df.text == "").sum()

1

In [26]:
print((df["label"] == "O").sum())
print(((df["label"] == "O") & df["text"].isna()).sum())

39258
29912
